In [1]:
import threading

In [16]:
threading.active_count(), threading.current_thread(), threading.enumerate()

(8,
 <_MainThread(MainThread, started 4725466560)>,
 [<_MainThread(MainThread, started 4725466560)>,
  <Thread(IOPub, started daemon 123145333313536)>,
  <Heartbeat(Heartbeat, started daemon 123145350103040)>,
  <Thread(Thread-3 (_watch_pipe_fd), started daemon 123145367965696)>,
  <Thread(Thread-4 (_watch_pipe_fd), started daemon 123145384755200)>,
  <ControlThread(Control, started daemon 123145401544704)>,
  <HistorySavingThread(IPythonHistorySavingThread, started 123145418334208)>,
  <ParentPollerUnix(Thread-2, started daemon 123145436196864)>])

In [6]:
threading.get_ident(), threading.get_native_id()

(4725466560, 381600)

In [29]:
N = 10 ** 6
TIMES = 100

def count(n):
    while n >= 0:
        n -= 1


def count_times(n, times=1):
    curr = threading.current_thread()
#     print(f"{threading.active_count()=}, {curr.name=}, {curr.native_id=}\n", end="")
    for _ in range(times):
        count(n)

In [10]:
%%time

count_times(N, TIMES)

CPU times: user 5.74 s, sys: 52.7 ms, total: 5.79 s
Wall time: 6.43 s


In [23]:
%%time

th = threading.Thread(target=count_times, name="th_1", args=(N, TIMES))
th.start()
th.join()

threading.active_count()=9, curr.name='th_1', curr.native_id=410576
CPU times: user 551 ms, sys: 5.66 ms, total: 556 ms
Wall time: 621 ms


In [30]:
%%time


threads = [
    threading.Thread(target=count_times, name=f"th_{i}", args=(N, 1))
    for i in range(TIMES)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 7.09 s, sys: 163 ms, total: 7.26 s
Wall time: 7.47 s


In [46]:
from urllib.request import urlopen

URL = "https://ru.wikipedia.org/wiki/Python"
URL_TIMES = 50


def fetch_url(url, results):
    resp = urlopen(url)
    results.append(resp.getcode())
    return resp


def fetch_url_times(url, results, times):
    for _ in range(times):
        fetch_url(url, results)
    return 99
    

In [37]:
r = fetch_url(URL)

In [39]:
r.getcode(), r.read().decode()

In [60]:
%%time

results = []

th = threading.Thread(target=fetch_url_times, name="th_1", args=(URL, results, URL_TIMES))

th.start()

th.join()

CPU times: user 742 ms, sys: 52.7 ms, total: 794 ms
Wall time: 8.35 s


In [61]:
%%time

import time

results = []

th = threading.Thread(target=fetch_url_times, name="th_1", args=(URL, results, URL_TIMES))

th.start()

while th.is_alive():
    print("awaitong&&&")
    time.sleep(1)
    

# th.join()

awaitong&&&
awaitong&&&
awaitong&&&
awaitong&&&
awaitong&&&
awaitong&&&
awaitong&&&
awaitong&&&
awaitong&&&
CPU times: user 794 ms, sys: 51.7 ms, total: 846 ms
Wall time: 9.06 s


In [59]:
%%time

results = []


threads = [
    threading.Thread(target=fetch_url_times, name=f"th_{i}", args=(URL, results, 1))
    for i in range(URL_TIMES)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
print(len(results), results)

50 [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
CPU times: user 814 ms, sys: 78.5 ms, total: 893 ms
Wall time: 893 ms


In [ ]:
socket.send(b"dwdwd")

In [51]:
class ResultThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.fetch_result = None
        
    def run(self):
        self.fetch_result = self._target(*self._args, **self._kwargs)

In [54]:
th = ResultThread(target=fetch_url_times, name="th_1", args=(URL, results, URL_TIMES))
th.start()
th.join()

th.fetch_result

99

In [62]:
%%time


URL = "https://ru.wikipedia.org/wiki/Python"
URL_TIMES = 50


def fetch_url(url, results):
    resp = urlopen(url)
    results.append(resp.getcode())
    return resp


def fetch_url_times(url, results, times, lock):
    for _ in range(times):
        lock.acquire()
        try:
            fetch_url(url, results)
        finally:
            lock.release()

    return 99
    

results = []
lock = threading.Lock()


threads = [
    threading.Thread(target=fetch_url_times, name=f"th_{i}", args=(URL, results, 1, lock))
    for i in range(URL_TIMES)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
print(len(results), results)

50 [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
CPU times: user 1.02 s, sys: 65.1 ms, total: 1.09 s
Wall time: 8.73 s


In [78]:
%%time


URL = "https://ru.wikipedia.org/wiki/Python"
URL_TIMES = 50


def fetch_url(url, results):
    resp = urlopen(url)
    results.append(resp.getcode())
    return resp


def fetch_url_times(url, results, times, lock):
    for _ in range(times):
        with lock:
            fetch_url(url, results)
#         lock.acquire()
#         try:
#             fetch_url(url, results)
#         finally:
#             lock.release()

    return 99
    

results = []
lock = threading.Semaphore(5)


threads = [
    threading.Thread(target=fetch_url_times, name=f"th_{i}", args=(URL, results, 1, lock))
    for i in range(URL_TIMES)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
print(len(results), results)

50 [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
CPU times: user 711 ms, sys: 100 ms, total: 812 ms
Wall time: 2.11 s


In [79]:
%%time

from queue import Queue
import threading


URL = "https://ru.wikipedia.org/wiki/Python"
URL_TIMES = 50



def fetch_url(url, results):
    resp = urlopen(url)
    results.append(resp.getcode())
    return resp


def fetch_url_times(url, results, times, q):
    while True:
        try:
            elem = q.get(timeout=1)
            print(f"COUNT {q.qsize()}\n", end="")
            fetch_url(elem, results)
            q.task_done()
        except Exception:
            pass
    

results = []
q = Queue(10)


threads = [
    threading.Thread(target=fetch_url_times, name=f"th_{i}", args=(URL, results, 1, q), daemon=True)
    for i in range(URL_TIMES)
]

for th in threads:
    th.start()
    
    
for _ in range(URL_TIMES):
    q.put(URL)


# for th in threads:
#     th.join()

q.join()
    
print(len(results), results)

COUNT 9
COUNT 8
COUNT 7
COUNT 6
COUNT 5
COUNT 4
COUNT 3
COUNT 2
COUNT 1
COUNT 9
COUNT 9
COUNT 8
COUNT 7
COUNT 6
COUNT 5
COUNT 4
COUNT 3
COUNT 2
COUNT 9
COUNT 8
COUNT 7
COUNT 6
COUNT 5
COUNT 9
COUNT 8
COUNT 7
COUNT 6
COUNT 5
COUNT 4
COUNT 3
COUNT 2
COUNT 1
COUNT 9
COUNT 8
COUNT 7
COUNT 6
COUNT 5
COUNT 4
COUNT 3
COUNT 2
COUNT 1
COUNT 8
COUNT 7
COUNT 6
COUNT 5
COUNT 4
COUNT 3
COUNT 2
COUNT 1
COUNT 0
50 [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
CPU times: user 593 ms, sys: 116 ms, total: 709 ms
Wall time: 636 ms
